# Comprehensive Educational Research Notebook

This notebook demonstrates a complete multi-agent research system that combines:

- **Full Research Workflow** (from `5_full_agent.ipynb`): Complete end-to-end research process
- **MCP Integration** (from `3_research_agent_mcp.ipynb`): Model Context Protocol for tool access
- **Test-Synchronized Examples** (from `0_consolidated_research_agent.ipynb`): Deterministic demonstrations

## Learning Objectives

By the end of this notebook, you will understand:

1. **System Architecture**: How multi-agent research systems are structured
2. **MCP Integration**: How to use Model Context Protocol for tool access
3. **LLM Impact**: How different prompts and LLM settings affect research quality
4. **Workflow Orchestration**: How LangGraph coordinates complex research workflows
5. **Test-Driven Development**: How to build reliable, testable AI systems

## Prerequisites

- Basic understanding of Python and async programming
- Familiarity with Jupyter notebooks
- Understanding of LLMs and their capabilities
- Basic knowledge of agent-based systems

## Notebook Structure

This notebook is organized into distinct sections, each building upon the previous:

1. **Bootstrap & Setup**: Environment configuration and initialization
2. **Core Components**: Understanding the building blocks
3. **MCP Integration**: Tool access and async operations
4. **Research Workflow**: Complete end-to-end process
5. **LLM Impact Analysis**: Understanding prompt and model effects
6. **Test Synchronization**: Ensuring reliability and reproducibility


## Section 1: Bootstrap & Setup

The first step in any robust AI system is proper initialization and configuration. This section demonstrates:

- **Environment Setup**: Loading configuration and setting up logging
- **Path Management**: Ensuring proper imports and module discovery
- **Bootstrap Process**: Initializing the research framework
- **Console Configuration**: Setting up rich output for educational purposes

### Why Bootstrap Matters

Bootstrap ensures that:
1. Environment variables are loaded correctly
2. Logging is configured for debugging and monitoring
3. Console output is formatted for readability
4. All dependencies are properly initialized
5. Error handling is set up with rich tracebacks


## TODO Anchors and Test Cross‑links

- [ ] Section 2: Core Components — see `tests/test_research_agent.py`
- [ ] Section 3: MCP Integration — see `tests/test_renderer_rich.py`
- [ ] Section 4: Research Workflow — see `tests/test_end_to_end_flow.py`
- [ ] Section 5: LLM Impact Analysis — see `tests/test_llm_mock.py`
- [ ] Section 6: Test Synchronization — see `tests/test_renderer.py`
- [ ] Search Adapters (SerpAPI/Tavily) — see `tests/test_serpapi_adapter.py`, `tests/test_tavily_adapter.py`, `tests/test_serpapi_and_tavily_adapters.py`, `tests/test_adapters*.py`
- [ ] Supervisor Policy Demo — see `tests/test_supervisor_policy.py`, `tests/test_supervisor_policy_deterministic.py`
- [ ] Bootstrap & Config Walkthrough — see `tests/test_bootstrap.py`, `tests/test_bootstrap_wiring.py`, `tests/test_config.py`


In [1]:
# Notebook helper: ensure bootstrap runs early and use centralized helpers
import sys
from pathlib import Path
from rich.console import Console

# Ensure local `src` is on sys.path so imports like `research_agent_framework` work
repo_cwd = Path.cwd().resolve()
found_src = None
for candidate in [repo_cwd] + list(repo_cwd.parents):
    if (candidate / "src" / "research_agent_framework").exists():
        found_src = (candidate / "src").resolve()
        break
if found_src is None:
    candidate = (repo_cwd / ".." / "src").resolve()
    if (candidate / "research_agent_framework").exists():
        found_src = candidate
if found_src is not None and str(found_src) not in sys.path:
    sys.path.insert(0, str(found_src))

# Import project bootstrap and helpers
from research_agent_framework.bootstrap import bootstrap
from research_agent_framework.config import get_settings, get_console, get_logger

# Initialize environment, console, and logging (idempotent)
bootstrap()

# Obtain shared handles via helpers
settings = get_settings()
console = get_console()
logger = get_logger()

def nb_console():
    """
    Return the project's shared `Console` instance via `get_console()`.
    This ensures consistent rich output formatting throughout the notebook.
    """
    try:
        return get_console()
    except Exception:
        return Console()


### Bootstrap Process Demonstration

Now let's run the bootstrap process and see what it initializes. This demonstrates how a production AI system should start up.


In [2]:
# Bootstrap the research framework
# This initializes logging, console, and environment configuration
from research_agent_framework.bootstrap import bootstrap
from research_agent_framework.config import get_settings, get_console, get_logger
from rich.panel import Panel
from rich.table import Table

# Run bootstrap to configure the environment
console.print("🔄 Running bootstrap process...")
bootstrap()

# Get the configured settings, console, and logger
settings = get_settings()
console = settings.console # get_console()
logger = settings.logger # get_logger()

# Display bootstrap information
console.print(Panel(
    "[bold green]✅ Bootstrap Complete[/bold green]\n\n"
    "The research framework has been initialized with:\n"
    "• Environment variables loaded\n"
    "• Logging configured (console sink)\n"
    "• Console formatting enabled\n"
    "• Error handling set up",
    title="Bootstrap Status",
    expand=False
))

# Show configuration details
config_table = Table(title="Framework Configuration", show_header=True, header_style="bold magenta")
config_table.add_column("Component", style="cyan", width=20)
config_table.add_column("Status", style="green", width=15)
config_table.add_column("Details", style="white", width=40)

config_table.add_row("Environment", "✅ Loaded", "Variables from .env file (if present)")
config_table.add_row("Logging", "✅ Configured", "Loguru wired to Rich Console")
config_table.add_row("Console", "✅ Ready", "Rich formatting enabled")
config_table.add_row("Error Handling", "✅ Active", "Rich tracebacks installed")

console.print(config_table)


🔄 Running bootstrap process...

╭──────────────── Bootstrap Status ─────────────────╮
│ ✅ Bootstrap Complete                             │
│                                                   │
│ The research framework has been initialized with: │
│ • Environment variables loaded                    │
│ • Logging configured (console sink)               │
│ • Console formatting enabled                      │
│ • Error handling set up                           │
╰───────────────────────────────────────────────────╯

                               Framework Configuration                               
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Component            ┃ Status          ┃ Details                                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Environment          │ ✅ Loaded       │ Variables from .env file (if present)    │
│ Logging              │ ✅ Configured   │ Loguru wired to Rich Console             │
│ Console              │ ✅ Ready        │ Rich formatting enabled                  │
│ Error Handling       │ ✅ Active       │ Rich tracebacks installed                │
└──────────────────────┴─────────────────┴──────────────────────────────────────────┘

### Configuration impact on behavior and logging

This section explains how changing settings (env vars) impacts runtime:

- `model_name` and `model_temperature` influence prompt behavior and deterministic outputs.
- `LOGGING__LEVEL` and `LOGGING__FMT` change the logging verbosity and format; the `Settings.logger` property reflects changes when `get_settings(force_reload=True)` is used.
- `enable_tracing` toggles optional tracing hooks (visualizations guarded by env).

Below is a safe example demonstrating how to reload settings at runtime and observe logger level changes without restarting the notebook.

Note: This example mutates process environment variables temporarily and reloads `Settings` with `force_reload=True` to illustrate effects in a deterministic demo.


In [3]:
# Demonstration: change logging level via env and reload settings
import os
from research_agent_framework.config import get_settings, get_logger

# Show current logging level
settings = get_settings()
print("Before reload: logging.level=", settings.logging.level)

# Temporarily set environment to DEBUG and reload
os.environ["LOGGING__LEVEL"] = "DEBUG"
settings = get_settings(force_reload=True)
print("After reload: logging.level=", settings.logging.level)

# Acquire logger and show that level reflects setting
logger = get_logger()
logger.info("This is an info message (should always show at INFO/DEBUG)")
logger.debug("This is a debug message (visible only when level=DEBUG)")

# Clean up: restore env and reload to original for deterministic notebook runs
os.environ.pop("LOGGING__LEVEL", None)
settings = get_settings(force_reload=True)
print("Restored: logging.level=", settings.logging.level)


Before reload: logging.level= INFO
After reload: logging.level= DEBUG


2025-09-12T08:53:43.982457-0700 INFO This is an info message (should always show at INFO/DEBUG)

2025-09-12T08:53:43.982457-0700 INFO This is an info message (should always show at INFO/DEBUG)
2025-09-12T08:53:43.982457-0700 INFO This is an info message (should always show at INFO/DEBUG)
2025-09-12T08:53:43.987291-0700 DEBUG This is a debug message (visible only when level=DEBUG)
2025-09-12T08:53:43.982457-0700 INFO This is an info message (should always show at INFO/DEBUG)
2025-09-12T08:53:43.987291-0700 DEBUG This is a debug message (visible only when level=DEBUG)


Restored: logging.level= INFO


## Architecture & Technologies (Brief Overview)

- **Settings & Bootstrap**: `Settings` (Pydantic) loads env; `bootstrap()` enables rich tracebacks and wires Loguru → Rich `Console`.
- **Logging**: `LoggingConfig` fields (`level`, `fmt`, `backend`) drive a lazy `logger` property; helpers delegate to the same instances.
- **Agents & Models**: Agents coordinate research steps; Pydantic models (`SerpResult`, `Scope`, etc.) provide typed state.
- **Adapters**: Search adapters (SerpAPI/Tavily) expose deterministic stubs with optional live paths.
- **Prompts/Renderer**: Jinja templates rendered with rich-markdown output for clarity.
- **Tests as Specs**: Notebook sections mirror `tests/` behaviors for deterministic, reproducible demos.


## Architecture Diagram (Components)

```mermaid
flowchart LR
  subgraph Agents
    A[Research Agent]
    S[Scoping Agent]
    SP[Supervisor]
  end

  subgraph Framework
    CFG[Settings]
    LCFG[LoggingConfig]
    CON[Console]
    LGR[Logger]
    PR[Prompt Renderer]
    LLM[LLM Client]
  end

  subgraph Adapters
    SERP[SerpAPI Adapter]
    TAV[Tavily Adapter]
  end

  A --> PR
  A --> LLM
  A --> SERP
  A --> TAV
  S --> PR
  SP --> A
  SP --> S

  CFG --> CON
  CFG --> LCFG
  LCFG --> LGR
  LGR --> CON
```


## Sequence Diagram (User → Scoping → Research → Synthesis → Report)

```mermaid
sequenceDiagram
    participant U as User
    participant S as Scoping Agent
    participant R as Research Agent
    participant L as LLM Client
    participant A as Search Adapter
    participant P as Prompt Renderer
    participant V as Supervisor

    U->>S: Provide initial question / constraints
    S->>P: Format scoping prompt
    P-->>S: Scoped questions / clarifications
    S->>U: Ask clarifying question (if needed)
    U-->>S: Clarified scope

    S->>R: Submit refined scope / research task
    R->>P: Render research prompt
    P-->>R: Prompt text
    R->>L: Query LLM for retrieval & analysis
    L-->>R: LLM response (results / suggestions)

    R->>A: Run search queries (documents, citations)
    A-->>R: Search results (Serp/Tavily)

    R->>R: Aggregate findings, synthesize insights
    R->>V: Hand-off for supervision / orchestration
    V-->>R: Supervisor decisions / reassignments

    R->>U: Deliver final report / brief
    Note over U,R: Report includes provenance and evidence links
```


## Data Model Diagram (Key models & relationships)

```mermaid
classDiagram
    direction LR
    class SerpResult {
        string id
        string title
        string snippet
        string url
        list citations
        +from_raw(dict) SerpResult
    }

    class Scope {
        string id
        string question
        list constraints
        list clarifications
    }

    class ResearchTask {
        string id
        Scope scope
        list steps
        status
    }

    class EvalResult {
        string task_id
        bool success
        float score
        string feedback
        dict details
    }

    class AgentContext {
        settings
        console
        logger
        llm_client
        search_adapter
    }

    SerpResult --|> ResearchTask : evidence
    Scope "1" o-- "0..*" ResearchTask : generates
    ResearchTask "1" o-- "0..*" EvalResult : evaluated_by
    AgentContext "1" -- "*" ResearchTask : used_by
```


### Why this setup (results of the design)

- **Env overrides work**: `LOGGING__LEVEL`, `LOGGING__FMT`, `LOGGING__BACKEND` populate real fields; the `logger` property reflects them at access time.
- **Single ownership via properties**: `settings.console` and `settings.logger` are the shared instances used everywhere.
- **Helpers remain simple**: `get_console()` / `get_logger()` just delegate to those shared instances when you prefer function calls.
- **Robust bootstrap**: `bootstrap()` installs rich tracebacks, ensures a Console, and wires Loguru → Console idempotently.
- **Notebook consistency**: Cells use property access (e.g., `settings.console`) for clarity; helpers are equivalent if preferred.


## Env vars: required and optional (4.1)

Key environment variables used by the framework (recommended defaults shown):

| Variable | Required? | Default | Description |
|---|---:|---|---|
| `SERPAPI_API_KEY` | Optional | — | API key for SerpAPI; if missing, notebook defaults to `Mock` adapter |
| `TAVILY_API_KEY` | Optional | — | API key for Tavily adapter; if missing, notebook defaults to `Mock` adapter |
| `LOGGING__LEVEL` | Optional | `INFO` | Logging verbosity |
| `LOGGING__FMT` | Optional | project default | Logging format string |
| `MODEL_NAME` | Optional | `mock-model` | LLM model to use when not mocking |
| `MODEL_TEMPERATURE` | Optional | `0.0` | Controls LLM sampling |
| `ENABLE_TRACING` | Optional | `False` | Enable tracing hooks |

This section lists recommended env vars, safe defaults, and guidance on toggling live providers vs mocks.

In [4]:
# Safe demo: display resolved settings and show defaults
from research_agent_framework.config import get_settings

s = get_settings(force_reload=True)
print('MODEL_NAME =', s.model_name)
print('MODEL_TEMPERATURE =', s.model_temperature)
print('LOGGING__LEVEL =', s.logging.level)
print('ENABLE_TRACING =', s.enable_tracing)

# Display whether external adapter keys are present
import os
print('SERPAPI_API_KEY present:', bool(os.environ.get('SERPAPI_API_KEY')))
print('TAVILY_API_KEY present:', bool(os.environ.get('TAVILY_API_KEY')))


MODEL_NAME = mock-model
MODEL_TEMPERATURE = 0.0
LOGGING__LEVEL = INFO
ENABLE_TRACING = False
SERPAPI_API_KEY present: False
TAVILY_API_KEY present: True


### Safe defaults and fallback behavior (4.2)

This cell demonstrates how the notebook and framework behave when external API keys are not provided. By default the notebook uses deterministic `Mock` adapters and `MockLLM` to keep examples reproducible and low-cost.

Key points:

- If `SERPAPI_API_KEY` or `TAVILY_API_KEY` are missing, the framework falls back to the `Mock` search adapter.
- If `MODEL_NAME` is set to a real provider name, `llm_factory` will create a live client — otherwise the `MockLLM` is used.
- Use `get_settings(force_reload=True)` after mutating `os.environ` to observe changes at runtime in an idempotent manner.

The next code cell runs a safe demo that temporarily unsets adapter-related env vars, reloads settings, and prints which adapters/LLM the framework would use.

In [5]:
# Demo: show which adapters/LLM would be used when adapter keys are absent
import os
from contextlib import contextmanager
from research_agent_framework.config import get_settings, get_console
from research_agent_framework.adapters.search import from_raw_adapter
from research_agent_framework.llm.client import llm_factory, LLMConfig, MockLLM

console = get_console()

@contextmanager
def temp_env_vars(*keys):
    """Temporarily pop keys from os.environ, restoring them on exit."""
    saved = {k: os.environ.pop(k, None) for k in keys}
    try:
        yield
    finally:
        for k, v in saved.items():
            if v is not None:
                os.environ[k] = v

# Show baseline
settings = get_settings(force_reload=True)
console.print(f"Baseline: MODEL_NAME={settings.model_name!r}, SERPAPI key present={bool(os.environ.get('SERPAPI_API_KEY'))}")

# Temporarily remove adapter keys to simulate missing credentials
with temp_env_vars('SERPAPI_API_KEY', 'TAVILY_API_KEY'):
    s = get_settings(force_reload=True)
    # Use the adapters package factory; provide an empty raw payload and explicit provider
    serp_adapter = from_raw_adapter({}, provider='serpapi')
    tav_adapter = from_raw_adapter({}, provider='tavily')

    # Build a minimal LLMConfig from settings and create an LLM client with fallback
    cfg = LLMConfig(api_key=s.llm_api_key or "", model=s.model_name or "mock-model", temperature=s.model_temperature)
    provider = 'mock' if (s.model_name is None or str(s.model_name).startswith('mock')) else str(s.model_name)
    try:
        llm = llm_factory(provider, cfg)
    except Exception:
        # Unknown provider or construction failure -> fallback to MockLLM
        llm = MockLLM(cfg)

    console.print(f"During missing-keys demo: serp_adapter={serp_adapter.__class__.__name__}, tav_adapter={tav_adapter.__class__.__name__}, llm={llm.__class__.__name__}")

# Restore and show
s2 = get_settings(force_reload=True)
console.print(f"After restore: MODEL_NAME={s2.model_name!r}")


Baseline: MODEL_NAME='mock-model', SERPAPI key present=False

During missing-keys demo: serp_adapter=SerpAPISearchAdapter, tav_adapter=TavilySearchAdapter, llm=MockLLM

After restore: MODEL_NAME='mock-model'

### Switchboard helper: centralize mock/live toggles (4.3)

This small utility centralizes environment-driven toggles used by the notebook and examples. Use the helper to make notebook cells short and declarative — change the environment in one place and the helper will consistently report whether the framework will use mocks or live providers.

The code cell below demonstrates toggling `FORCE_USE_MOCK` and observing the resolved behavior for search adapters and the LLM.

In [6]:
# Switchboard demo cell: toggle FORCE_USE_MOCK and show effective choices
import os
from research_agent_framework.helpers.switchboard import use_mock_search, use_mock_llm
from research_agent_framework.config import get_console, get_settings

console = get_console()

# Baseline
s = get_settings(force_reload=True)
console.print(f"Baseline: use_mock_search={use_mock_search(s)}, use_mock_llm={use_mock_llm(s)}")

# Force use of mocks
os.environ['FORCE_USE_MOCK'] = '1'
s_forced = get_settings(force_reload=True)
console.print(f"After FORCE_USE_MOCK=1: use_mock_search={use_mock_search(s_forced)}, use_mock_llm={use_mock_llm(s_forced)}")

# Clean up
os.environ.pop('FORCE_USE_MOCK', None)
s_restored = get_settings(force_reload=True)
console.print(f"After restore: use_mock_search={use_mock_search(s_restored)}, use_mock_llm={use_mock_llm(s_restored)}")


Baseline: use_mock_search=False, use_mock_llm=True

After FORCE_USE_MOCK=1: use_mock_search=True, use_mock_llm=True

After restore: use_mock_search=False, use_mock_llm=True

### Central switchboard (6.0) - single place to toggle mocks vs live providers

This small, editable cell is the recommended place to toggle the environment for the entire notebook when you want to run the examples against live providers. By default the notebook is mock-first (safe and deterministic).

Change `FORCE_USE_MOCK` below or set provider-specific keys (`SERPAPI_API_KEY`, `TAVILY_API_KEY`, `MODEL_NAME`) to run with live services. Use `get_settings(force_reload=True)` after editing to apply changes in subsequent cells.

In [ ]:
# Central switchboard: edit this cell to toggle mock vs live globally for the notebook
# Options:
#  - Set FORCE_USE_MOCK=1 to force all mocks
#  - Unset FORCE_USE_MOCK and set provider keys (SERPAPI_API_KEY/TAVILY_API_KEY/MODEL_NAME) to use live providers
import os
# Example: force mocks for all demo cells (safe default)
os.environ['FORCE_USE_MOCK'] = os.environ.get('FORCE_USE_MOCK', '1')
# Example: to use live providers, uncomment and set real keys here (DO NOT commit secrets)
# os.environ.pop('FORCE_USE_MOCK', None)
# os.environ['SERPAPI_API_KEY'] = 'sk-...your-key...'
# os.environ['TAVILY_API_KEY'] = 'tk-...your-key...'
# os.environ['MODEL_NAME'] = 'openai-gpt-4'

# Apply settings reload so later cells observe the new environment
from research_agent_framework.config import get_settings
s = get_settings(force_reload=True)
print('Central switchboard applied. Current: FORCE_USE_MOCK=', os.environ.get('FORCE_USE_MOCK'), 'MODEL_NAME=', s.model_name)
